In [1]:
import numpy as np
#from ptsa.data.filters import MorletWaveletFilter
from ptsa.data.filters.MorletWaveletFilter import MorletWaveletFilter
from ptsa.data.readers import EEGReader
from ptsa.data.filters import MonopolarToBipolarMapper
from ptsa.data.filters import ResampleFilter
from ptsa.data.filters import ButterworthFilter
# from ptsa.data.readers.TalReader import TalReader
from ptsa.data.readers import TalReader
# from ptsa.data.readers.IndexReader import JsonIndexReader
from ptsa.data.readers import JsonIndexReader
from ptsa.data.readers import BaseEventReader
from ptsa.data.TimeSeriesX import TimeSeriesX as TimeSeries
import h5py
#from exceptions import OSError
import os
import mne
from glob import glob




In [2]:
rhino_mount = ''
jr = JsonIndexReader(rhino_mount + '/protocols/ltp.json')
exp = 'ltpFR2'

In [4]:
evs = []

bad_subjsess_files = [
    rhino_mount+
    '/protocols/ltp/subjects/LTP326/experiments/ltpFR2/sessions/19/behavioral/current_processed/task_events.json']

for f in sorted(jr.aggregate_values('task_events', experiment=exp))[-10:]:
    if f in bad_subjsess_files:
        continue
    tmpev = BaseEventReader(filename=f).read()
    if len(tmpev) == 0:
        continue
    # only use BioSemi events
    if tmpev['eegfile'][0].split('.')[-1] != 'bdf':
        continue
    else:
        # only use encoding events:
        tmpev = tmpev[tmpev['type'] == 'WORD']
        evs.append(tmpev)
evs = np.concatenate(evs)

evs['eegfile'] = np.array([rhino_mount+ef for ef in evs['eegfile']])


In [5]:
channels = ['A'+str(i) for i in range(1, 33)]
channels.extend(['B'+str(i) for i in range(1, 33)])
channels.extend(['C'+str(i) for i in range(1, 33)])
channels.extend(['D'+str(i) for i in range(1, 33)])

channels = np.array(channels)
channels

array(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11',
       'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20',
       'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29',
       'A30', 'A31', 'A32', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
       'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17',
       'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26',
       'B27', 'B28', 'B29', 'B30', 'B31', 'B32', 'C1', 'C2', 'C3', 'C4',
       'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23',
       'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32',
       'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11',
       'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20',
       'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29',
       'D30', 'D31', 'D32'], dtype='<U3')

In [6]:
start_time = -0.2
end_time = 1.0
buf_time = 2.0
powhz = 50
freqs = np.logspace(np.log10(2), np.log10(200), 15)

In [7]:
def get_raw_eeg(subj, sess):
    eegpath = (rhino_mount+'/protocols/ltp/subjects/'+subj+
               '/experiments/ltpFR2/sessions/'+str(sess)+
               '/ephys/current_processed/')
    sys = 'bio' if int(subj[-3:]) > 330 else 'egi'
    if sys == 'bio':
        eegfile = glob(eegpath+'*.bdf')
        if len(eegfile)!=1:
            return
        eegfile = eegfile[0]
        raw = mne.io.read_raw_edf(eegfile, eog=['EXG1', 'EXG2', 'EXG3', 'EXG4'], 
                                  misc=['EXG5', 'EXG6', 'EXG7', 'EXG8'], 
                                  stim_channel='Status', 
                                  montage='biosemi128', 
                                  preload=True) # needs to be true for 0.1Hz high-pass filter to work

    else:
        # for now catch this case -- different pipeline for EGI subjects!
        raise ValueError('EGI subject!')
        eegfile = (glob(eegpath+'*.2.raw') + glob(eegpath+'*.1.raw') +
                   glob(eegpath+'*.mff'))
        if len(eegfile)!=1:
            return
        eegfile = eegfile[0]
        raw = mne.io.read_raw_egi(eegfile, preload=True)
        raw.rename_channels({'E129': 'Cz'})
        raw.set_montage(mne.channels.read_montage('GSN-HydroCel-129'))
        raw.set_channel_types({'E8': 'eog', 'E25': 'eog', 'E126': 'eog',
                               'E127': 'eog', 'Cz': 'misc'})
        
    # get bad channels - new version
    badchanfile = glob(eegpath+'*_bad_chan.txt')
    if len(badchanfile) > 0:
        with open(badchanfile[0], 'r') as f:
            bad = [s.strip() for s in f.readlines()]
        raw.info['bads'] = bad
    # 0.1Hz high-pass filter
    raw.filter(l_freq=0.1, h_freq=None, method='iir', iir_params=None)
    return raw

In [8]:
subj = evs['subject'][0]
sess = evs['session'][0]

In [16]:
dat=get_raw_eeg(subj, sess)

Extracting EDF parameters from /protocols/ltp/subjects/LTP393/experiments/ltpFR2/sessions/21/ephys/current_processed/LTP393_session_21.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 10651647  =      0.000 ...  5201.000 secs...
Setting up high-pass filter at 0.1 Hz


In [17]:
def create_epoch(raw, events, start_time=start_time,
                 end_time=end_time, buf_time=buf_time):
    mne_events = np.zeros((len(events), 3), dtype=int)
    mne_events[:, 0] = events['eegoffset']
    ep = mne.Epochs(raw, mne_events, tmin=start_time-buf_time,
                    tmax=end_time+buf_time, baseline=None, preload=True)
    # convert to microvolts, average reference
    ep._data = ep._data * 1000000
    ep.set_eeg_reference('average', projection=False)

    ep.pick_types(eeg=True, exclude=[])
    # # - resample
    # ep.resample(500.0)
    return ep



In [18]:
subjfilt = evs['subject'] == subj
sessfilt = evs['session'] == sess
dat = create_epoch(dat, events=evs[100:105],
                   start_time=start_time,
                   end_time=end_time, buf_time=buf_time)
dat = TimeSeries(dat._data, dims=('events','channels','time'),
                 coords={'events': evs[100:105], #[subjfilt & sessfilt], 
                         'channels': dat.info['ch_names'], 
                         'time': dat.times,
                         'samplerate': dat.info['sfreq']})


5 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 5 events and 10651 original time points ...
0 bad epochs dropped
Applying average reference.
Applying a custom EEG reference.


In [19]:
chan = 'A23'
dat = dat.sel(channels=[chan])
ev_subjsess = evs[subjfilt & sessfilt]

In [22]:
dat -= dat.mean('time')
dat.mean('time')

<xarray.TimeSeriesX (events: 5, channels: 1)>
array([[ 1.091640e-14],
       [-4.058219e-15],
       [ 3.971828e-16],
       [ 8.813405e-16],
       [ 1.585396e-15]])
Coordinates:
  * events      (events) (numpy.record, [('begin_distractor', '<i8'), ('begin_math_correct', '<i8'), ('eegfile', '<U256'), ('eegoffset', '<i8'), ('eogArtifact', '<i8'), ('experiment', '<U256'), ('final_distractor', '<i8'), ('final_math_correct', '<i8'), ('intruded', '<i8'), ('intrusion', '<i8'), ('item_name', '<U256'), ('item_num', '<i8'), ('montage', '<U256'), ('msoffset', '<i8'), ('mstime', '<i8'), ('protocol', '<U256'), ('recalled', '<i8'), ('rectime', '<i8'), ('serialpos', '<i8'), ('session', '<i8'), ('subject', '<U256'), ('trial', '<i8'), ('type', '<U256')]) (0, -999, '/protocols/ltp/subjects/LTP393/experiments/ltpFR2/sessions/21/ephys/current_processed/LTP393_session_21.bdf', 2142975, 0, 'ltpFR2', 24000, 9, 0, -999, 'BARREL', 91, '0.0', 1, 1526485083826, 'ltp', 0, -999, 5, 21, 'LTP393', 5, 'WORD') ...
 

In [25]:
dat = MorletWaveletFilter(
            # timeseries=dat_chan, freqs=freqs, output='power', width=5,
            time_series=dat, freqs=freqs, output='power', width=5,
            verbose=True).filter()[0]  # ['power']
dat = np.log10(dat)


total time wavelet loop:  0.06775879859924316


TypeError: __init__() takes 1 positional argument but 2 were given

In [28]:
dat

<xarray.DataArray (frequency: 15, events: 5, channels: 1, time: 10651)>
array([[[[7.063094, ..., 5.949063]],

        ...,

        [[5.89553 , ..., 4.943697]]],


       ...,


       [[[4.768149, ..., 3.86895 ]],

        ...,

        [[4.633956, ..., 4.773765]]]], dtype=float32)
Coordinates:
  * events      (events) (numpy.record, [('begin_distractor', '<i8'), ('begin_math_correct', '<i8'), ('eegfile', '<U256'), ('eegoffset', '<i8'), ('eogArtifact', '<i8'), ('experiment', '<U256'), ('final_distractor', '<i8'), ('final_math_correct', '<i8'), ('intruded', '<i8'), ('intrusion', '<i8'), ('item_name', '<U256'), ('item_num', '<i8'), ('montage', '<U256'), ('msoffset', '<i8'), ('mstime', '<i8'), ('protocol', '<U256'), ('recalled', '<i8'), ('rectime', '<i8'), ('serialpos', '<i8'), ('session', '<i8'), ('subject', '<U256'), ('trial', '<i8'), ('type', '<U256')]) (0, -999, '/protocols/ltp/subjects/LTP393/experiments/ltpFR2/sessions/21/ephys/current_processed/LTP393_session_21.bdf', 2142975, 0, 

In [35]:
powhz = 50.
dat = TimeSeries(dat, coords=dat.coords)
dat = ResampleFilter(time_series=dat, resamplerate=powhz).filter()
#dat = dat.remove_buffer(duration=buf_time)
# dat.to_hdf(pfile_path, compression='gzip', compression_opts=9, mode='w-')

260


In [36]:
dat

<xarray.TimeSeriesX (frequency: 15, events: 5, channels: 1, time: 260)>
array([[[[6.527364, ..., 5.885956]],

        ...,

        [[5.43626 , ..., 4.894592]]],


       ...,


       [[[4.561391, ..., 4.286065]],

        ...,

        [[4.854352, ..., 4.679119]]]])
Coordinates:
  * frequency   (frequency) float64 2.0 2.779 3.861 5.365 7.455 10.36 14.39 ...
  * events      (events) (numpy.record, [('begin_distractor', '<i8'), ('begin_math_correct', '<i8'), ('eegfile', '<U256'), ('eegoffset', '<i8'), ('eogArtifact', '<i8'), ('experiment', '<U256'), ('final_distractor', '<i8'), ('final_math_correct', '<i8'), ('intruded', '<i8'), ('intrusion', '<i8'), ('item_name', '<U256'), ('item_num', '<i8'), ('montage', '<U256'), ('msoffset', '<i8'), ('mstime', '<i8'), ('protocol', '<U256'), ('recalled', '<i8'), ('rectime', '<i8'), ('serialpos', '<i8'), ('session', '<i8'), ('subject', '<U256'), ('trial', '<i8'), ('type', '<U256')]) (0, -999, '/protocols/ltp/subjects/LTP393/experiments/ltpFR2/session

In [37]:
test = np.arange(100)

In [39]:
test[::2]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66,
       68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98])

In [40]:
test

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [41]:
dat['frequency']

<xarray.TimeSeriesX 'frequency' (frequency: 15)>
array([  2.      ,   2.778991,   3.861395,   5.365392,   7.455187,  10.358949,
        14.393713,  20.      ,  27.78991 ,  38.613955,  53.653916,  74.551874,
       103.589494, 143.937135, 200.      ])
Coordinates:
  * frequency   (frequency) float64 2.0 2.779 3.861 5.365 7.455 10.36 14.39 ...
    samplerate  float64 50.0

In [44]:
0.05 / (200*128*15)

1.3020833333333334e-07

In [45]:
dat

<xarray.TimeSeriesX (frequency: 15, events: 5, channels: 1, time: 260)>
array([[[[6.527364, ..., 5.885956]],

        ...,

        [[5.43626 , ..., 4.894592]]],


       ...,


       [[[4.561391, ..., 4.286065]],

        ...,

        [[4.854352, ..., 4.679119]]]])
Coordinates:
  * frequency   (frequency) float64 2.0 2.779 3.861 5.365 7.455 10.36 14.39 ...
  * events      (events) (numpy.record, [('begin_distractor', '<i8'), ('begin_math_correct', '<i8'), ('eegfile', '<U256'), ('eegoffset', '<i8'), ('eogArtifact', '<i8'), ('experiment', '<U256'), ('final_distractor', '<i8'), ('final_math_correct', '<i8'), ('intruded', '<i8'), ('intrusion', '<i8'), ('item_name', '<U256'), ('item_num', '<i8'), ('montage', '<U256'), ('msoffset', '<i8'), ('mstime', '<i8'), ('protocol', '<U256'), ('recalled', '<i8'), ('rectime', '<i8'), ('serialpos', '<i8'), ('session', '<i8'), ('subject', '<U256'), ('trial', '<i8'), ('type', '<U256')]) (0, -999, '/protocols/ltp/subjects/LTP393/experiments/ltpFR2/session